In [1]:
import sys
import os
import shutil

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from custum_dataset import digi_data_load, testcase_add_data_load, testcase_shuffle_data_load
from attn_bi_lstm import ABLSTM

In [6]:
df = pd.read_csv('../../nlu-model-analysis/binary_dataset.csv',sep='\t',index_col='Unnamed: 0')
df = df.sample(len(df),random_state=10)
df['class'] = df['class'].apply(lambda x: x if x== 1 else 0 )
df

,sent,class
24879,디스플레이,0
33870,로우,0
104066,코렐,0
8224,쿠폰 안되나요,1
6641,쿠폰요,1
101506,헤어장식,0
89576,현금영수증 발행이 필요해요,1
93839,bh6083,0
6959,담에 하겠습니다,1
33793,알려주세요,0


In [7]:
train_df = df.iloc[:int(len(df)*0.8)]
test_df = df.iloc[int(len(df)*0.8):]

In [8]:
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = train_df['sent'], train_df['class'], test_df['sent'], test_df['class']

In [9]:
train_intent = TRAIN_LABEL
test_intent = TEST_LABEL

In [10]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), test_doc)))

CPU times: user 235 ms, sys: 13.9 ms, total: 249 ms
Wall time: 251 ms


In [11]:
y_train = pd.Series(train_intent.tolist())
y_test = pd.Series(test_intent.tolist())

In [12]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=64)
vocab_size = tokenizer.num_words

In [13]:
tokenizer.word_docs

{'플': 878,
 '이': 18457,
 '스': 6841,
 '레': 1484,
 '디': 2779,
 '로': 6714,
 '우': 1820,
 '렐': 20,
 '코': 950,
 '폰': 3697,
 'ⓢ': 52623,
 '되': 4566,
 '쿠': 3220,
 '나': 8175,
 '요': 22137,
 '안': 5265,
 '헤': 262,
 '장': 3556,
 '어': 12477,
 '식': 907,
 '증': 1099,
 '행': 774,
 '수': 4232,
 '영': 539,
 '금': 2371,
 '해': 6880,
 '필': 751,
 '발': 1000,
 '현': 539,
 'b': 904,
 '0': 5287,
 '3': 2718,
 '6': 1897,
 '8': 1942,
 'h': 1236,
 '다': 8135,
 '에': 6687,
 '하': 9381,
 '니': 7587,
 '습': 1773,
 '겠': 576,
 '담': 1356,
 '주': 6737,
 '알': 1957,
 '려': 3244,
 '세': 4819,
 '치': 2188,
 '약': 482,
 '방': 1993,
 '가': 12256,
 '동': 1412,
 '아': 8703,
 '닥': 95,
 '것': 1040,
 '탈': 254,
 '여': 3017,
 '곳': 169,
 '뭐': 1242,
 '는': 11737,
 '기': 8156,
 '무': 1994,
 'n': 2668,
 '고': 10140,
 '줄': 296,
 '씨': 321,
 '오': 3938,
 'ㅓ': 70,
 '옹': 42,
 '성': 1935,
 '결': 2078,
 '청': 2719,
 '제': 7681,
 '양': 626,
 '손': 165,
 '톱': 26,
 '교': 744,
 '떻': 2150,
 '게': 4486,
 '환': 2425,
 '은': 5570,
 '구': 5439,
 '르': 692,
 '유': 1803,
 '트': 6072,
 '위': 522,
 '시

In [14]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 64,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 2,
    "learning_rate": 0.00012, #1e-3,
    "decay": 0.90,
    "batch_size": 512,
    "train_epoch": 20
}


Validation Size:  21560


In [15]:
modelname = 'binary_classifier'

try:
    shutil.rmtree('./output/'+modelname)
except OSError as e:
    if e.errno == 2:
        # 파일이나 디렉토리가 없음!
        print ('No such file or directory to remove')
        pass
    else:
        raise

In [16]:
sess = tf.Session()

classifier = ABLSTM(sess, config)
classifier.build_graph(modelname)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

resume = False
if resume:
    saver.restore(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

dev_batch = (x_dev, y_dev)
start = time.time()
step = 0
lr = config['learning_rate']
for e in range(config["train_epoch"]):
    
    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch),lr)
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch), lr)
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
        if step % 10 == 0:
            feed_dict = make_test_feed_dict(classifier, (x_batch, y_batch))
            tr_merged = sess.run(classifier.merge, feed_dict)
            feed_dict = make_test_feed_dict(classifier, (x_test, y_test))
            ts_merged = sess.run(classifier.merge, feed_dict)
            classifier.train_writer.add_summary(tr_merged, step)
            classifier.test_writer.add_summary(ts_merged, step)
            
        step += 1
    t1 = time.time()
    if e+1 % 10: 
        lr = lr*config['decay']
    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

building graph
graph built successfully!
Epoch 1 start !
Train Epoch time:  280.744 s
validation accuracy: 0.761 
Epoch 2 start !
Train Epoch time:  274.324 s
validation accuracy: 0.816 
Epoch 3 start !
Train Epoch time:  274.417 s
validation accuracy: 0.839 
Epoch 4 start !
Train Epoch time:  274.313 s
validation accuracy: 0.850 
Epoch 5 start !
Train Epoch time:  263.266 s
validation accuracy: 0.850 
Epoch 6 start !
Train Epoch time:  274.643 s
validation accuracy: 0.859 
Epoch 7 start !
Train Epoch time:  273.098 s
validation accuracy: 0.863 
Epoch 8 start !
Train Epoch time:  273.595 s
validation accuracy: 0.866 
Epoch 9 start !
Train Epoch time:  272.490 s
validation accuracy: 0.868 
Epoch 10 start !
Train Epoch time:  264.116 s
validation accuracy: 0.871 
Epoch 11 start !
Train Epoch time:  274.078 s
validation accuracy: 0.871 
Epoch 12 start !
Train Epoch time:  274.614 s
validation accuracy: 0.874 
Epoch 13 start !
Train Epoch time:  274.665 s
validation accuracy: 0.874 
Epoch 

In [17]:
if "Saver" not in os.listdir("./"):
    os.makedirs("./Saver")
if modelname not in os.listdir("./Saver"):
    os.makedirs("./Saver/{}".format(modelname))
saver.save(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

'./Saver/binary_classifier/binary_classifier.ckpt'

In [18]:
print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))

Training finished, time consumed :  5620.205155849457  s
Start evaluating:  

Test accuracy : 87.662760 %
